In [1]:
import findspark
findspark.init('/home/ecomog/spark-2.4.3-bin-hadoop2.7')
import pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
# create a spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()